In [ ]:
import os
import sys

# 프로젝트 루트(이 노트북의 상위 디렉터리)를 파이썬 모듈 검색 경로에 추가
# (`notebooks/` 폴더 안에 있고, 상위 폴더에 `graph.py`가 있다는 전제)
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_dir not in sys.path:
    sys.path.append(root_dir)

from graph import create_chat_graph

# 안전을 위해 API 키는 환경변수로 관리하는 것을 권장합니다.
# 예: 터미널에서 `export GOOGLE_API_KEY=...` 후 아래처럼 사용
api_key = os.getenv("GOOGLE_API_KEY", "")

chat_graph = create_chat_graph(
    api_key=api_key,
    model_name="gemini-2.5-flash"
)

In [ ]:
chat_graph

예시 1: stream()을 사용한 State History

[초기 State]
{
  "messages": [
    "content='안녕하세요! 반갑습니다.' additional_kwargs={} response_metadata={}"
  ]
}

[실행 중 State 변화]

--- Step 1 ---
{
  "chatbot": {
    "messages": [
      "content='안녕하세요! 저도 반갑습니다. 😊\\n\\n무엇을 도와드릴까요? 편하게 말씀해주세요!' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'} id='lc_run--019bbc38-bccd-7cd3-be3b-23e8411f8e0e-0' tool_calls=[] invalid_tool_calls=[] usage_metadata={'input_tokens': 5, 'output_tokens': 323, 'total_tokens': 328, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 303}}"
    ]
  }
}

최종 결과 (invoke 사용)
{
  "messages": [
    "content='안녕하세요! 반갑습니다.' additional_kwargs={} response_metadata={} id='fd8c8acb-16a8-407e-9286-8992ea9e12ea'",
    "content='안녕하세요! 저도 반갑습니다. 😊\\n\\n무엇을 도와드릴까요? 편하게 말씀해주세요!' additional_kwargs={} response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5

In [12]:
from langchain_core.messages import HumanMessage

chat_graph.invoke({"messages": [HumanMessage(content="안녕하세요")]})

{'messages': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}, id='6895f6fe-f115-424a-9c73-205db897d3d6'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bbc3a-b2e3-7040-b380-d7016d8453b9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2, 'output_tokens': 182, 'total_tokens': 184, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 174}})]}

In [14]:
# Config를 사용한 State History 추적
# MemorySaver를 사용하여 state를 저장하고 thread_id로 관리

from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from graph import State
import json
import uuid

print("=" * 60)
print("Config를 사용한 State History 추적")
print("=" * 60)

# MemorySaver를 사용하여 그래프 생성하는 함수
def create_chat_graph_with_memory(api_key: str, model_name: str = "gemini-pro"):
    """MemorySaver를 사용하는 채팅 그래프 생성"""
    llm = ChatGoogleGenerativeAI(
        model=model_name,
        google_api_key=api_key,
        temperature=0.7,
    )
    
    def chatbot(state: State):
        """채팅봇 응답 생성"""
        response = llm.invoke(state["messages"])
        return {"messages": [response]}
    
    # 그래프 구성
    graph = StateGraph(State)
    graph.add_node("chatbot", chatbot)
    graph.set_entry_point("chatbot")
    graph.add_edge("chatbot", END)
    
    # MemorySaver를 사용하여 컴파일
    memory = MemorySaver()
    return graph.compile(checkpointer=memory)

# MemorySaver를 사용하는 그래프 생성
chat_graph_with_memory = create_chat_graph_with_memory(api_key, "gemini-2.5-flash")

print("\n✅ MemorySaver를 사용하는 그래프가 생성되었습니다!")
print(f"   그래프 타입: {type(chat_graph_with_memory).__name__}")

Config를 사용한 State History 추적

✅ MemorySaver를 사용하는 그래프가 생성되었습니다!
   그래프 타입: CompiledStateGraph


In [16]:
# Thread ID를 사용하여 대화 세션 관리
print("\n" + "=" * 60)
print("Thread ID를 사용한 State History 관리")
print("=" * 60)

# 고유한 thread_id 생성
thread_id = str(uuid.uuid4())
print(f"\n[Thread ID 생성]")
print(f"  thread_id: {thread_id}")

# Config 설정 (thread_id 포함)
config = {"configurable": {"thread_id": thread_id}}

# 첫 번째 메시지 전송
print(f"\n[첫 번째 메시지 전송]")
result1 = chat_graph_with_memory.invoke(
    {"messages": [HumanMessage(content="안녕하세요! 제 이름은 홍길동입니다.")]},
    config=config
)

print(f"  응답: {result1['messages'][-1].content[:100]}...")

# 현재 state 조회
current_state = chat_graph_with_memory.get_state(config)
print(f"\n[현재 State 조회]")
print(f"  메시지 개수: {len(current_state.values['messages'])}")
print(f"  State 값:")
for i, msg in enumerate(current_state.values['messages'], 1):
    msg_type = type(msg).__name__
    content = msg.content[:80] if hasattr(msg, 'content') else str(msg)[:80]
    print(f"    {i}. [{msg_type}] {content}...")


Thread ID를 사용한 State History 관리

[Thread ID 생성]
  thread_id: 4a3643a5-01c1-4ff5-95ec-af2ac3c00825

[첫 번째 메시지 전송]
  응답: 안녕하세요, 홍길동님! 만나서 반갑습니다. 😊...

[현재 State 조회]
  메시지 개수: 2
  State 값:
    1. [HumanMessage] 안녕하세요! 제 이름은 홍길동입니다....
    2. [AIMessage] 안녕하세요, 홍길동님! 만나서 반갑습니다. 😊...


In [17]:
# 같은 thread_id로 계속 대화 (State가 자동으로 유지됨)
print("\n" + "=" * 60)
print("같은 Thread ID로 대화 계속하기")
print("=" * 60)

# 두 번째 메시지 전송 (같은 config 사용)
print(f"\n[두 번째 메시지 전송]")
result2 = chat_graph_with_memory.invoke(
    {"messages": [HumanMessage(content="제 이름이 뭐였죠?")]},
    config=config  # 같은 thread_id 사용
)

print(f"  응답: {result2['messages'][-1].content[:200]}...")

# State 확인 (이전 대화가 포함되어 있음)
updated_state = chat_graph_with_memory.get_state(config)
print(f"\n[업데이트된 State 조회]")
print(f"  메시지 개수: {len(updated_state.values['messages'])}")
print(f"  전체 대화 히스토리:")
for i, msg in enumerate(updated_state.values['messages'], 1):
    msg_type = type(msg).__name__
    content = msg.content[:150] if hasattr(msg, 'content') else str(msg)[:150]
    print(f"\n    {i}. [{msg_type}]")
    print(f"       {content}...")


같은 Thread ID로 대화 계속하기

[두 번째 메시지 전송]
  응답: 홍길동님이십니다!...

[업데이트된 State 조회]
  메시지 개수: 4
  전체 대화 히스토리:

    1. [HumanMessage]
       안녕하세요! 제 이름은 홍길동입니다....

    2. [AIMessage]
       안녕하세요, 홍길동님! 만나서 반갑습니다. 😊...

    3. [HumanMessage]
       제 이름이 뭐였죠?...

    4. [AIMessage]
       홍길동님이십니다!...


In [27]:
# 여러 Thread를 관리하고 각각의 State 조회
print("\n" + "=" * 60)
print("여러 Thread 관리 및 State 조회")
print("=" * 60)

# 여러 thread 생성
threads = {}
for i in range(3):
    thread_id = str(uuid.uuid4())
    threads[f"thread_{i+1}"] = thread_id
    
    config = {"configurable": {"thread_id": thread_id}}
    
    # 각 thread에 메시지 전송
    result = chat_graph_with_memory.invoke(
        {"messages": [HumanMessage(content=f"안녕하세요! 저는 사용자 {i+1}입니다.")]},
        config=config
    )
    
    print(f"\n[Thread {i+1}]")
    print(f"  Thread ID: {thread_id}")
    print(f"  응답: {result['messages'][-1].content[:100]}...")

# 모든 thread의 state 조회
print("\n" + "=" * 60)
print("모든 Thread의 State 조회")
print("=" * 60)

for name, tid in threads.items():
    config = {"configurable": {"thread_id": tid}}
    state = chat_graph_with_memory.get_state(config)
    print(f"\n[{name}]")
    print(f"  Thread ID: {tid}")
    print(f"  메시지 개수: {len(state.values['messages'])}")
    print(f"  마지막 메시지: {state.values['messages'][-1].content[:80]}...")


여러 Thread 관리 및 State 조회

[Thread 1]
  Thread ID: 2aeac6d2-37f1-4a2a-89bc-26945bfc6748
  응답: 안녕하세요, 사용자 1님! 만나서 반갑습니다. 😊

무엇을 도와드릴까요? 편하게 말씀해주세요!...

[Thread 2]
  Thread ID: 7a30de37-2d43-4172-9b7e-59255ebc7915
  응답: 안녕하세요, 사용자 2님! 만나서 반갑습니다. 😊

무엇을 도와드릴까요? 어떤 이야기를 나누고 싶으신가요?...

[Thread 3]
  Thread ID: c64bf14c-e2d8-4120-b2c5-67439ef5de90
  응답: 안녕하세요, 사용자 3님! 만나서 반갑습니다.

무엇을 도와드릴까요? 어떤 궁금한 점이 있으신가요?...

모든 Thread의 State 조회

[thread_1]
  Thread ID: 2aeac6d2-37f1-4a2a-89bc-26945bfc6748
  메시지 개수: 2
  마지막 메시지: 안녕하세요, 사용자 1님! 만나서 반갑습니다. 😊

무엇을 도와드릴까요? 편하게 말씀해주세요!...

[thread_2]
  Thread ID: 7a30de37-2d43-4172-9b7e-59255ebc7915
  메시지 개수: 2
  마지막 메시지: 안녕하세요, 사용자 2님! 만나서 반갑습니다. 😊

무엇을 도와드릴까요? 어떤 이야기를 나누고 싶으신가요?...

[thread_3]
  Thread ID: c64bf14c-e2d8-4120-b2c5-67439ef5de90
  메시지 개수: 2
  마지막 메시지: 안녕하세요, 사용자 3님! 만나서 반갑습니다.

무엇을 도와드릴까요? 어떤 궁금한 점이 있으신가요?...


In [28]:
# 특정 Thread의 State를 업데이트하거나 초기화
print("\n" + "=" * 60)
print("State 업데이트 및 초기화")
print("=" * 60)

# 첫 번째 thread 선택
test_thread_id = list(threads.values())[0]
test_config = {"configurable": {"thread_id": test_thread_id}}

print(f"\n[선택된 Thread]")
print(f"  Thread ID: {test_thread_id}")

# 현재 state 확인
current_state = chat_graph_with_memory.get_state(test_config)
print(f"\n[현재 State]")
print(f"  메시지 개수: {len(current_state.values['messages'])}")

# State를 직접 업데이트 (예: 메시지 추가)
# 주의: 일반적으로는 invoke를 통해 state를 업데이트하는 것이 권장됩니다
print(f"\n[State 업데이트 방법]")
print(f"  1. invoke() 사용: 새로운 메시지를 추가하면 자동으로 state가 업데이트됩니다")
print(f"  2. update_state() 사용: state를 직접 수정할 수 있습니다")

# 새로운 메시지로 state 업데이트
new_result = chat_graph_with_memory.invoke(
    {"messages": [HumanMessage(content="추가 질문입니다.")]},
    config=test_config
)

updated_state = chat_graph_with_memory.get_state(test_config)
print(f"\n[업데이트 후 State]")
print(f"  메시지 개수: {len(updated_state.values['messages'])}")
print(f"  전체 대화:")
for i, msg in enumerate(updated_state.values['messages'], 1):
    msg_type = type(msg).__name__
    content = msg.content[:100] if hasattr(msg, 'content') else str(msg)[:100]
    print(f"    {i}. [{msg_type}] {content}...")


State 업데이트 및 초기화

[선택된 Thread]
  Thread ID: 2aeac6d2-37f1-4a2a-89bc-26945bfc6748

[현재 State]
  메시지 개수: 2

[State 업데이트 방법]
  1. invoke() 사용: 새로운 메시지를 추가하면 자동으로 state가 업데이트됩니다
  2. update_state() 사용: state를 직접 수정할 수 있습니다

[업데이트 후 State]
  메시지 개수: 4
  전체 대화:
    1. [HumanMessage] 안녕하세요! 저는 사용자 1입니다....
    2. [AIMessage] 안녕하세요, 사용자 1님! 만나서 반갑습니다. 😊

무엇을 도와드릴까요? 편하게 말씀해주세요!...
    3. [HumanMessage] 추가 질문입니다....
    4. [AIMessage] 네, 좋습니다! 추가 질문이 있으시군요.

무엇이든 편하게 여쭤보세요! 기다리고 있겠습니다. 😊...


In [29]:
# 요약: Config를 사용한 State History 관리 패턴
print("\n" + "=" * 60)
print("Config를 사용한 State History 관리 요약")
print("=" * 60)

summary = """
[주요 개념]

1. MemorySaver: State를 메모리에 저장하는 체크포인터
   - 그래프 컴파일 시 checkpointer=MemorySaver() 사용

2. Config: 실행 시 thread_id를 포함한 설정
   - config = {"configurable": {"thread_id": "..."}}
   - 같은 thread_id를 사용하면 state가 유지됨

3. 주요 메서드:
   - get_state(config): 현재 state 조회
   - invoke(input, config): state를 업데이트하며 실행
   - update_state(config, values): state 직접 업데이트 (고급)

[사용 패턴]

1. 그래프 생성:
   memory = MemorySaver()
   graph = graph.compile(checkpointer=memory)

2. Thread 생성 및 사용:
   thread_id = str(uuid.uuid4())
   config = {"configurable": {"thread_id": thread_id}}
   result = graph.invoke(input, config=config)

3. State 조회:
   state = graph.get_state(config)
   messages = state.values['messages']

4. 대화 계속하기:
   # 같은 config를 사용하면 이전 대화가 자동으로 포함됨
   result = graph.invoke(new_input, config=config)
"""

print(summary)


Config를 사용한 State History 관리 요약

[주요 개념]

1. MemorySaver: State를 메모리에 저장하는 체크포인터
   - 그래프 컴파일 시 checkpointer=MemorySaver() 사용

2. Config: 실행 시 thread_id를 포함한 설정
   - config = {"configurable": {"thread_id": "..."}}
   - 같은 thread_id를 사용하면 state가 유지됨

3. 주요 메서드:
   - get_state(config): 현재 state 조회
   - invoke(input, config): state를 업데이트하며 실행
   - update_state(config, values): state 직접 업데이트 (고급)

[사용 패턴]

1. 그래프 생성:
   memory = MemorySaver()
   graph = graph.compile(checkpointer=memory)

2. Thread 생성 및 사용:
   thread_id = str(uuid.uuid4())
   config = {"configurable": {"thread_id": thread_id}}
   result = graph.invoke(input, config=config)

3. State 조회:
   state = graph.get_state(config)
   messages = state.values['messages']

4. 대화 계속하기:
   # 같은 config를 사용하면 이전 대화가 자동으로 포함됨
   result = graph.invoke(new_input, config=config)

